In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import math
import re
import warnings
import textwrap

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# Read gene information
genes = r"path/genes.csv"
genes = pd.read_csv(genes)

# Read masks
msk = r"path/files/mask_def.tsv"
msk = pd.read_csv(msk, sep="\t", header=None)

# Read and prepare list of binary phenotypes
phen_b = r"path/pheno_binary.txt"
pheno_b = pd.read_csv(phen_b, header=None)
pheno_b[0] = pheno_b[0].str.rsplit('.',n=1).str.get(0)

# Read and prepare list of quantitative phenotypes
phen_q = r"path/pheno_quant.txt"
pheno_q = pd.read_csv(phen_q, header=None)
pheno_q[0]=pheno_q[0].str.rsplit('.',n=1).str.get(0)


In [4]:
def get_pos(s):
    '''
    function for getting position from snp string
    '''
    if isinstance(s, str) == False:
        return None
    if len(s.split(':'))<2:
        return None
    return int(s.split(':')[1])

In [5]:
def get_full_inf(pheno, df, phen_type:str, j):
    """
    Function for adding some information (midpoint pos, phenotype, mask name) to df.
    -----
    pheno: str
        the phenotype file to read
    df: pandas.DataFrame
        df to add results to
    phen_type:str
        binary or quantitative (for folder address and to fill column)
    j: int
        chromosome
    ------------
    return pd.DataFrame
    """
 
    for i in range(len(pheno)):
        # Get phenotype value        
        phen = pheno[0].values[i]

        # Read regenie output file for chromosome by chromosome and phenotype 
        rege = r"path/"+phen_type+"/"+phen+"/step2_"+str(j)+"_"+phen+".regenie"
        reg = pd.read_csv(rege, sep=" ", skiprows=1)
        
        # Add phenotype column to the regenie output file      
        reg['pheno'] = phen
        reg['pheno_type'] = phen_type

        # Remove all summary rows and reset index
        reg = reg.dropna(subset=['ALLELE0'])
        reg = reg.reset_index()

        # Read snplist file
        snpl = r"path/"+phen_type+"/"+phen+"/step2_"+str(j)+"_masks.snplist"
        snps = pd.read_csv(snpl, sep="\t", header=None)

        # Add info from file to regenie df (number of vars and start and end pos for each row)
        snps[1] = snps[1].str.split(',').apply(lambda x: [k for k in x])
        reg['enp_s'] = snps[1].apply(lambda x: x[0]).apply(get_pos)
        reg['snp_e'] = snps[1].apply(lambda x: x[-1]).apply(get_pos)
        reg['pos'] = reg['enp_s'] + (reg['snp_e'] - reg['enp_s'])//2
        
        # Add mask name        
        reg['mask']=''
        mask = reg['ALLELE1'].str.rsplit('.',n=2).str.get(0)
        for i in range(len(reg)):
            if mask.iloc[i]=='Mask1':
                reg['mask'].iloc[i] = msk[1].iloc[0]
            elif mask.iloc[i]=='Mask2':
                reg['mask'].iloc[i] = msk[1].iloc[1]
            elif mask.iloc[i]=='Mask3':
                reg['mask'].iloc[i] = msk[1].iloc[2]
            elif mask.iloc[i]=='Mask4':
                reg['mask'].iloc[i] = msk[1].iloc[3]
            elif mask.iloc[i]=='Mask5':
                reg['mask'].iloc[i] = msk[1].iloc[4]
        
        
        # Add significant results (log10 p-val > cut-off) to the sig df    
        df = pd.concat([df, reg])
        
        
    return df
        

In [6]:
def nearest_genes(chrom, pos, genes, distance_le):
    """
    A function for finding the nearest gene to a given
    chromosome position.
    
    ---------------------
    chrom int or str:
        The chromosome number
    pos int:
        The position on the chromosome
    gene pandas.DataFrame:
        Reference dataset showing type and position of genes.
    distance_le int:
        Maximum the distance to search from our position.
    
    ---------------------
    filt.iloc[0] pandas.core.series.Series: 
        A pandas series containing the information of the 
        nearest gene.
    pos_st str:
        The start position for search.
    pos_en str:
        The end position for search.
        
    """
    pos_st = pos - distance_le
    pos_en = pos + distance_le

    filt = genes[genes['genetype']=="protein_coding"]
    filt=filt.dropna(subset=['hgncid'])
    filt = filt[filt['chrom']==str(chrom)]
    filt = filt[filt['pos_end']>=pos_st]
    filt = filt[filt['pos_start']<=pos_en]

    if(len(filt)==0):
        return None, str(pos_st), str(pos_en)
    
    filt['dist2gene'] = 0.0
    for j in range(len(filt)):
        if filt['pos_start'].iloc[j] >= pos:  
            filt['dist2gene'].iloc[j] = float(filt['pos_start'].iloc[j] - pos)
        elif filt['pos_end'].iloc[j] >= pos and filt['pos_start'].iloc[j] <= pos:
            filt['dist2gene'].iloc[j] = 0
        elif filt['pos_end'].iloc[j] <= pos:
            filt['dist2gene'].iloc[j] = float(pos - filt['pos_end'].iloc[j])    
    
    filt = filt.sort_values(by=['dist2gene'])
    return filt.iloc[0], str(pos_st), str(pos_en)


In [7]:
def snp_inf(pheno, reg, phen_type:str):
    '''
    Fill more snp info.
    '''
    for j in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X']:
        for i in range(len(pheno)):
            # Get phenotype value        
            phen = pheno[0].values[i]

            # Read snplist file
            snpl = r"/path/"+phen_type+"/"+phen+"/step2_"+str(j)+"_masks.snplist"
            snps = pd.read_csv(snpl, sep="\t", header=None)

            # Add info from file to regenie df (number of vars and start and end pos for each row)
            snps[1] = snps[1].str.split(',').apply(lambda x: [k for k in x])
            snps['num_var'] = snps[1].apply(len)
            snps['snp_start_pos'] = snps[1].apply(lambda x: x[0])
            snps['snp_end_pos'] = snps[1].apply(lambda x: x[-1])
            snps = snps.rename(columns={0:'ID'})
            snps = snps.drop([1],axis=1)
            snps = snps.set_index('ID')
            reg['num_var']=reg['ID'].map(snps['num_var']).fillna(reg['num_var'])
            reg['snp_start_pos']=reg['ID'].map(snps['snp_start_pos']).fillna(reg['snp_start_pos'])
            reg['snp_end_pos']=reg['ID'].map(snps['snp_end_pos']).fillna(reg['snp_end_pos'])
            snps = None
        
    return reg
        

In [8]:
def other_inf(sig):
    '''
    Function for getting some other info from existing columns and logs.
    '''
    # This cell is for adding column to sig to use for plots
    sig['CHROM']=sig['CHROM'].replace(23, 'X')
    sig['CHROM']=sig['CHROM'].replace(23.0, 'X')

    # Init columns

    sig['Al1']=''
    sig['OR']=0
    sig['upper']=0
    sig['lower']=0
    sig['MAF']=0
    sig['ID2']=''
    sig['cases']=0
    sig['controls']=0

    # Init column based values (sub strings)
    ID = sig['ID'].str.rsplit('.',n=3).str.get(0)
    maf = sig['ALLELE1'].str.split('.',n=1).str.get(1)
    for i in range(len(sig)):
        # Fill other initialized columns   
        sig['OR'].iloc[i] = math.exp(sig['BETA'].iloc[i])
        sig['upper'].iloc[i] = math.exp(sig['BETA'].iloc[i]+(1.96*sig['SE'].iloc[i]))
        sig['lower'].iloc[i] = math.exp(sig['BETA'].iloc[i]-(1.96*sig['SE'].iloc[i]))
        sig['MAF'].iloc[i] = maf.iloc[i]
        sig['Al1'].iloc[i] = sig['mask'].iloc[i] + '_' + str(sig['MAF'].iloc[i])
        sig['ID2'].iloc[i] = ID.iloc[i]

        # Get regenie log url
        phe = sig['pheno'].iloc[i]
        if sig['CHROM'].iloc[i]!='X':
            chro = str(int(sig['CHROM'].iloc[i]))
        else:
            chro = 'X'
        ty = sig['pheno_type'].iloc[i]
        log = "/path/"+ty+"/"+phe+"/step2_"+chro+".log"

        # Read log info and get the number of cases and controls if binary
        if ty =='binary':
            with open(log, 'rb') as f:
                contents = f.read().decode('utf-8')
                num_cases = int(re.search(r'(\d+) cases', contents).group(1))
                num_controls = int(re.search(r'(\d+) controls', contents).group(1))
        else:
            with open(log, 'rb') as f:
                contents = f.read().decode('utf-8')
                num_cases = int(re.search(r'(\d+) observations', contents).group(1))
                num_controls = ''

        sig['cases'].iloc[i] = num_cases
        sig['controls'].iloc[i] = num_controls
        
    return sig


In [11]:
# To read sig df if needed
sig = r"./path/all_sigs.tsv"
sig = pd.read_csv(sig, sep="\t")

def conv(x):
    try:
        return int(float(x))
    except ValueError:
        return x
    
sig['CHROM'] = sig['CHROM'].apply(conv)

In [12]:
def near_gen(sig):
    '''
    Function for getting nearest protein-coding gene.
    '''
    sig['pos_st']=''
    sig['pos_en']=''
    sig['ensembl_gene_id']='N/A'
    sig['gene_name']='N/A'

    for i in range(len(sig)):
        gene_f, sig['pos_st'].iloc[i], sig['pos_en'].iloc[i] = nearest_genes(sig['CHROM'].iloc[i], 
                                                                             sig['GENPOS'].iloc[i], 
                                                                             genes, 8e5)
        if (gene_f is not None):
            sig['ensembl_gene_id'].iloc[i] = gene_f['ensembl_gene_id']
            sig['gene_name'].iloc[i] = gene_f['gene_name']
            
    return sig


In [13]:
def coor(sig):
    '''
    Function for getting coordinates  for manhattan plot.
    '''
    sig['pos'] = sig['pos'].astype(int)
    sig = sig.sort_values(by=['pos'])
    x_coor = []
    cur_chrom=None
    cur_pos=None
    cur_x=0
    for i, row in sig.iterrows():
        if row['CHROM']!= cur_chrom:
            cur_x+=1
            cur_pos=row['pos']
            cur_chrom=row['CHROM']
        elif row['pos']-cur_pos>100000:
            cur_x+=1
            cur_pos=row['pos']
        x_coor.append(cur_x)

    sig['x_coor']= x_coor
    return sig


In [3]:
# read files for chrom and add info for binary phenos
reg = None
for chrom in range(1,23):
    reg = None
    reg = get_full_inf(pheno_b, reg, 'binary', int(chrom))
    reg.to_csv("./path/binary"+str(chrom)+".tsv", index=False, sep='\t')
    print(chrom)

reg = None
chrom='X'
reg = get_full_inf(pheno_b, reg, 'binary', chrom)
reg.to_csv("./path/binary"+str(chrom)+".tsv", index=False, sep='\t')
print(chrom)

In [4]:
# read files for chrom and add info for quantitative phenos
reg = None
chrom='X'
reg = get_full_inf(pheno_q, reg, 'quan', chrom)
reg.to_csv("./path/quan"+str(chrom)+".tsv", index=False, sep='\t')
print(chrom)


reg = None
for chrom in range(1,23):
    reg = None
    reg = get_full_inf(pheno_q, reg, 'quan', int(chrom))
    reg.to_csv("./path/quan"+str(chrom)+".tsv", index=False, sep='\t')
    print(chrom)



In [2]:
# read in binary and quantitative info for chrom and merge and save

for chrom in range(1,23):
    reg = pd.read_csv("./path/binary"+str(chrom)+".tsv", sep="\t")
    print(len(reg))
    reg2 = pd.read_csv("./path/quan"+str(chrom)+".tsv", sep="\t")
    print(len(reg2))
    reg = reg.append(reg2).reset_index(drop=True).drop(['index'],axis=1)
    print('app')
    reg.to_csv("./path/"+str(chrom)+".tsv", index=False, sep='\t')
    print(chrom)
    reg=None
    reg2=None
    
chrom='X'
reg = pd.read_csv("./path/binary"+str(chrom)+".tsv", sep="\t")
print(len(reg))
reg2 = pd.read_csv("./path/quan"+str(chrom)+".tsv", sep="\t")
print(len(reg2))
reg = reg.append(reg2).reset_index(drop=True).drop(['index'],axis=1)
print('app')
reg.to_csv("./path/"+str(chrom)+".tsv", index=False, sep='\t')
print(chrom)
reg=None
reg2=None

In [5]:
# get coor for all chroms
add = 0
for chrom in range(1,23):
    reg = None
    rege = r"./path/"+str(chrom)+".tsv"
    size=1000000

    it = pd.read_csv(rege,chunksize=size, sep='\t')
    for chunk in it:
        reg = pd.concat([reg, chunk])
        print(len(reg))
    
    reg = coor(reg)
    print('coor')
    reg['x_coor']=reg['x_coor'] + add
    add = reg['x_coor'].iloc[-1]
    print('add')
    reg.to_csv("./path/co"+str(chrom)+".tsv", index=False, sep='\t')
    print(chrom) 
    print(add)
    
    
chrom = 'X'
reg = None
rege = r"./path/"+str(chrom)+".tsv"
size=1000000

it = pd.read_csv(rege,chunksize=size, sep='\t')
for chunk in it:
    reg = pd.concat([reg, chunk])
    print(len(reg))
        
reg = coor(reg)
print('coor')
reg['x_coor']=reg['x_coor'] + add
add = reg['x_coor'].iloc[-1]
print('add')
reg.to_csv("./path/co"+str(chrom)+".tsv", index=False, sep='\t')

In [6]:
# make and save sig for each chrom
for chrom in range(1,23):
    reg = None
    rege = r"./path/co"+str(chrom)+".tsv"
    size=1000000

    it = pd.read_csv(rege,chunksize=size, sep='\t')
    for chunk in it:
        reg = pd.concat([reg, chunk[chunk['LOG10P']>5]])
        print(len(reg))
        
    reg = near_gen(reg)
    print('near gene')
    reg = other_inf(reg)
    print('other inf')
    
    reg.to_csv("./path/sig"+str(chrom)+".tsv", index=False, sep='\t')
    print(chrom)
    
chrom = 'X'
reg = None
rege = r"./path/co"+str(chrom)+".tsv"
size=1000000

it = pd.read_csv(rege,chunksize=size, sep='\t')
for chunk in it:
    reg = pd.concat([reg, chunk[chunk['LOG10P']>5]])
    print(len(reg))
        
reg['CHROM'] = 'X'
reg = near_gen(reg)
print('near gene')
reg = other_inf(reg)
print('other inf')
    
reg.to_csv("./path/sig"+str(chrom)+".tsv", index=False, sep='\t')
print(chrom)

In [7]:
# Add all sigs to one files and save
reg = None
reg2 = None
for chrom in range(1,23):
    reg2 = pd.read_csv("./path/sig"+str(chrom)+".tsv", sep="\t")
    reg = pd.concat([reg, reg2])
    print(chrom)
    reg2=None

chrom='X'
reg2 = pd.read_csv("./path/sig"+str(chrom)+".tsv", sep="\t")
reg = pd.concat([reg, reg2])
print(chrom)
reg2=None    

reg.to_csv("./path/all_sigs.tsv", index=False, sep='\t')

In [19]:
# fill in snp info for sig and save final version
sig['num_var'] = ''
sig['snp_start_pos'] = ''
sig['snp_end_pos'] = ''
sig=snp_inf(pheno_q, sig, 'quan')
sig=snp_inf(pheno_b, sig, 'binary')
sig.to_csv("./path/full_sigs.tsv", index=False, sep='\t')

In [9]:
# read last file in if needed
sig = r"./path/full_sigs.tsv"
sig = pd.read_csv(sig, sep="\t")

def conv(x):
    try:
        return int(float(x))
    except ValueError:
        return x
    
sig['CHROM'] = sig['CHROM'].apply(conv)